In [1]:
!nvidia-smi

Fri Jun 11 12:20:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers

     |████████████████████████████████| 2.3MB 4.0MB/s 
     |████████████████████████████████| 3.3MB 50.4MB/s 
     |████████████████████████████████| 901kB 50.2MB/s 


In [3]:
import time
import torch
from transformers import GPTNeoForCausalLM
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B").cuda()

In [2]:
print(model.dtype, model.device)
print("seq_len\tmax_len\truntime\tmemory")
for seq_len in [64, 127, 251, 256, 383, 509, 512, 769, 1021, 1024, 1280, 1283, 1536, 1543, 1792, 1801, 2039, 2048]:
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()
  ids = torch.zeros(1, max(1, seq_len)).long().cuda()
  runtime = 0.
  max_length = min(2049, seq_len + 40)
  for i in range(10):
    s = time.perf_counter()
    outputs = model.generate(ids, use_cache=True, do_sample=True, min_length=max_length, max_length=max_length, pad_token_id=50256)
    runtime += time.perf_counter() - s
    del outputs
  print(f"{seq_len}\t{max_length}\t{runtime / 10.:.4f}s\t{torch.cuda.max_memory_allocated() /1024. / 1024. / 1024.:.3f}GB")
  del ids

torch.float32 cuda:0
seq_len	max_len	runtime	memory
64	104	1.4277s	10.171GB
127	167	1.7245s	10.234GB
251	291	2.1584s	10.361GB
256	296	2.1812s	10.362GB
383	423	2.9463s	11.219GB
509	549	3.4149s	11.595GB
512	552	2.9872s	10.651GB
769	809	4.9036s	12.375GB


RuntimeError: ignored

In [3]:
import transformers
transformers.__version__

'4.6.1'

In [1]:
# restarted runtime after oom and ran this with the model in half precision
import time
import torch
from transformers import GPTNeoForCausalLM
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B").half().cuda()

print(model.dtype, model.device)
print("seq_len\tmax_len\truntime\tmemory")
for seq_len in [64, 127, 251, 256, 383, 509, 512, 769, 1021, 1024, 1280, 1283, 1536, 1543, 1792, 1801, 2039, 2048]:
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()
  ids = torch.zeros(1, max(1, seq_len)).long().cuda()
  runtime = 0.
  max_length = min(2049, seq_len + 40)
  for i in range(10):
    s = time.perf_counter()
    outputs = model.generate(ids, use_cache=True, do_sample=True, min_length=max_length, max_length=max_length, pad_token_id=50256)
    runtime += time.perf_counter() - s
    del outputs
  print(f"{seq_len}\t{max_length}\t{runtime / 10.:.4f}s\t{torch.cuda.max_memory_allocated() /1024. / 1024. / 1024.:.3f}GB")
  del ids

torch.float16 cuda:0
seq_len	max_len	runtime	memory
64	104	1.2820s	5.059GB
127	167	1.4140s	5.090GB
251	291	1.8570s	5.181GB
256	296	1.8761s	5.180GB
383	423	2.8156s	6.994GB
509	549	3.3879s	7.646GB
512	552	2.7486s	5.317GB
769	809	4.6178s	9.000GB
1021	1061	5.7816s	10.304GB
1024	1064	4.4912s	5.590GB
1280	1320	5.3699s	5.735GB
1283	1323	7.0542s	11.664GB
1536	1576	6.2600s	5.882GB
1543	1583	8.2814s	13.013GB
1792	1832	7.1836s	6.023GB
1801	1841	9.5767s	14.350GB


RuntimeError: ignored

In [1]:
# to make sure it's not an issue with the GPU, restarted runtime and tried with the gpt-j branch
# see the other notebook for a full run: https://github.com/finetuneanon/misc/blob/main/transformers_benchmark/localattention_gpt-j.ipynb
!pip uninstall transformers
!pip install git+https://github.com/finetuneanon/transformers@gpt-j
import time
import torch
from transformers import GPTNeoForCausalLM
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B").half().cuda()

print(model.dtype, model.device)
print("seq_len\tmax_len\truntime\tmemory")
for seq_len in [64, 127, 251, 256]:
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()
  ids = torch.zeros(1, max(1, seq_len)).long().cuda()
  runtime = 0.
  max_length = min(2049, seq_len + 40)
  for i in range(10):
    s = time.perf_counter()
    outputs = model.generate(ids, use_cache=True, do_sample=True, min_length=max_length, max_length=max_length, pad_token_id=50256)
    runtime += time.perf_counter() - s
    del outputs
  print(f"{seq_len}\t{max_length}\t{runtime / 10.:.4f}s\t{torch.cuda.max_memory_allocated() /1024. / 1024. / 1024.:.3f}GB")
  del ids

Uninstalling transformers-4.6.1:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.7/dist-packages/transformers-4.6.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/transformers/*
Proceed (y/n)? y
  Successfully uninstalled transformers-4.6.1
  Cloning https://github.com/finetuneanon/transformers (to revision gpt-j) to /tmp/pip-req-build-u134983i
  Running command git clone -q https://github.com/finetuneanon/transformers /tmp/pip-req-build-u134983i
  Running command git checkout -b gpt-j --track origin/gpt-j
  Switched to a new branch 'gpt-j'
  Branch 'gpt-j' set up to track remote branch 'gpt-j' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.7.0.dev0-cp37-none-any.whl size=2405405 sha256=f077fca685f822c575e391c4bb0615ad1e7a1596f1e19bb164186669c00cfea4
  Stored in directory: /tmp/pip-ephem-wheel-cach

In [ ]:
# roughly matches the results from the other notebook, so should not be due to hardware differences